In [2]:
!pip install torchaudio
!pip install transformers
!pip install datasets
!pip install lang_trans
!pip install arabic_reshaper
!pip install python-bidi
!pip install pydub
!pip install soundfile
!pip install jiwer
!pip install PyArabic

     |████████████████████████████████| 2.9 MB 2.6 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.3 MB/s eta 0:00:38tcmalloc: large alloc 1147494400 bytes == 0x564eab682000 @  0x7f3bbf8ae615 0x564e714754cc 0x564e7155547a 0x564e714782ed 0x564e71569e1d 0x564e714ebe99 0x564e714e69ee 0x564e71479bda 0x564e714ebd00 0x564e714e69ee 0x564e71479bda 0x564e714e8737 0x564e7156ac66 0x564e714e7daf 0x564e7156ac66 0x564e714e7daf 0x564e7156ac66 0x564e714e7daf 0x564e7147a039 0x564e714bd409 0x564e71478c52 0x564e714ebc25 0x564e714e69ee 0x564e71479bda 0x564e714e8737 0x564e714e69ee 0x564e71479bda 0x564e714e7915 0x564e71479afa 0x564e714e7c0d 0x564e714e69ee
     |████████████████████████████████| 881.9 MB 13 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu111
    Uninstalling torch-1.9.0+cu111:
      Successfully uninstalled torch-1.9.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is 

In [3]:
import os
import re
import sys
import json
import torch
import jiwer
import logging
import librosa
import datasets
import itertools
import torchaudio
import numpy as np
import transformers
import pandas as pd
from torch import nn
import seaborn as sns
import torch.nn as nn
import soundfile as sf
import arabic_reshaper
from pyarabic import araby
from packaging import version
from pydub import AudioSegment
import matplotlib.pyplot as plt
from pydub.utils import mediainfo
from argparse import ArgumentParser
from torch.nn import functional as F
from contextlib import contextmanager
from bidi.algorithm import get_display
from lang_trans.arabic import buckwalter
from dataclasses import dataclass, field
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score
from typing import Any, Dict, List, Optional, Union
from transformers import HfArgumentParser,TrainingArguments
from sklearn.metrics import classification_report, confusion_matrix
from transformers.trainer_utils import get_last_checkpoint, is_main_process
from transformers import is_apex_available,set_seed ,Trainer,Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Model,Wav2Vec2PreTrainedModel
% matplotlib inline

In [4]:
!cp -r /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/All_poems.zip .

In [ ]:
if os.path.exists('dataset'):
  if len(os.listdir('dataset')) == 0:
    os.system('unzip All_poems.zip -d dataset')
else:
  os.system('unzip All_poems.zip -d dataset')

In [ ]:
!mkdir -p dataset_wav

In [ ]:
!cp /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/testset.csv .
!cp /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/trainset.csv .
!cp /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/valset.csv .

In [ ]:
metadata_test_path = 'testset.csv'
metadata_train_path = 'trainset.csv'
metadata_val_path = 'valset.csv'
dataset_folder = 'dataset'
dataset_wav_folder = 'dataset_wav'

In [ ]:
train_metadata = pd.read_csv(metadata_train_path)
test_metadata = pd.read_csv(metadata_test_path)
val_metadata = pd.read_csv(metadata_val_path)

In [ ]:
sample_rates = set()
for file_path in itertools.chain(
    train_metadata['Utterance name'],
    test_metadata['Utterance name'],
    val_metadata['Utterance name']
  ):
  complete_path = f'{dataset_folder}/{file_path}'
  complete_wav_path = f'{dataset_wav_folder}/{file_path}'
  # os.system(f'ffmpeg -i {complete_path} {complete_wav_path}')
  audio = AudioSegment.from_file(complete_path)
  sample_rates.add(audio.frame_rate)
  audio.export(f'{dataset_wav_folder}/{file_path}', format='wav')
sample_rates

{44100, 48000}

In [ ]:
# the following can be used
# https://beta.quod.ai/github/huggingface/transformers?question_modal=true&question_public_id=8757&from_search=true